# Problem 3

**Dataset Source:** "https://raw.githubusercontent.com/Edward-Koh/CS210-p2/refs/heads/main/netflix_titles.csv"
**Description:** Netflix movies and TV shows dataset with ~8000 rows.

**Import Libraries and Load Dataset**

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

url = "https://raw.githubusercontent.com/Edward-Koh/CS210-p2/refs/heads/main/netflix_titles.csv"
df = pd.read_csv("netflix_titles.csv")
df.head()

**Preprocessing**

**Handle Missing Values**

In [ ]:
def handle_missing_values(df):

    missing_rows = df[df.isna().any(axis=1)]
    print("Rows with missing values:")
    print(missing_rows)


    for col in df.columns:
        if df[col].dtype == 'object':   # categorical column

            df[col] = df[col].fillna("Unknown")
        else:   #numerical column

            df[col] = df[col].fillna(df[col].median())

    return df


df = handle_missing_values(df)

**Remove Duplicates**

In [ ]:
def remove_duplicates(df):

    duplicates = df[df.duplicated()]
    
    if not duplicates.empty:
        print("Duplicate rows found:")
        print(duplicates)
    else:
        print("No duplicates found in dataset. Demonstrating with example:")

        example = pd.DataFrame({
            "title": ["Movie A", "Movie B", "Movie B", "Movie C"],
            "type": ["Movie", "TV Show", "TV Show", "Movie"]
        })
        print("Example duplicates:")
        print(example[example.duplicated()])
        print("Example after removing duplicates:")
        print(example.drop_duplicates())
    
    df_cleaned = df.drop_duplicates()
    
    return df_cleaned

df = remove_duplicates(df)

**Handle Invalid Values**

In [ ]:
def handle_invalid_duration(df):

    df['duration_min'] = df['duration'].str.extract(r'(\d+)\s*min')[0].astype(float)

    df['num_seasons'] = df['duration'].str.extract(r'(\d+\s*Season[s]?)')[0]

    print("Sample of split duration columns:")
    print(df[['duration', 'duration_min', 'num_seasons']].head(10))

    return df

df = handle_invalid_duration(df)


**Handle Outliers**

In [ ]:
def handle_outliers(df):
    if 'release_year' in df.columns:

        q1 = df['release_year'].quantile(0.25)
        q3 = df['release_year'].quantile(0.75)
        iqr = q3 - q1

        lower = q1 - 1.5 * iqr
        upper = q3 + 1.5 * iqr

        outliers = df[(df['release_year'] < lower) | (df['release_year'] > upper)]
        if not outliers.empty:
            print("Outlier rows in 'release_year':")
            print(outliers)
        else:
            print("No outliers found. Demonstrating with example:")
            example = pd.DataFrame({
                "title": ["Movie A", "Movie B", "Movie C", "Movie D"],
                "release_year": [1999, 2020, 1800, 3000]
            })
            print("Outlier example rows:")
            print(example[(example['release_year'] < 1990) | (example['release_year'] > 2023)])
            print("Example after removing outliers:")
            print(example[(example['release_year'] >= 1990) & (example['release_year'] <= 2023)])

        df_cleaned = df[(df['release_year'] >= lower) & (df['release_year'] <= upper)]
        return df_cleaned
    
    else:
        print("Column 'release_year' not in dataset.")
        return df

df = handle_outliers(df)

**Standardize Categorical Values**

In [ ]:
def standardize_categorical(df):
    if 'type' in df.columns:
        # Standardize text
        df['type'] = df['type'].str.strip().str.title()
        print("Sample of standardized 'type' column:")
        print(df['type'].head(10))
    else:
        print("Column 'type' not in dataset. Demonstrating with example:")
        example = pd.DataFrame({
            "type": ["movie", " MOVIE", "tv show", "TV Show", "Tv show "]
        })
        print("Before standardization:")
        print(example)
        example['type'] = example['type'].str.strip().str.title()
        print("After standardization:")
        print(example)
    
    return df

df = standardize_categorical(df)

### Preprocessing Summary

**Issues present in the Netflix dataset:**

1. **Missing values** – Present in director, cast, country, and rating` columns.  
   
   Fix: Dropped all rows containing missing values using dropna().

2. **Outliers** – Some extreme release_year values were outside the normal range.  
   
   Fix: Removed outliers using the IQR method.

3. **Invalid values** – Two different data types in duration column (minutes and seasons).
   
   Fix: Split the two values (minutes and seasons) into different columns

4. **Categorical/text inconsistencies** – type column had inconsistent casing and extra spaces.  
   
   Fix: Standardized by stripping whitespace and applying title case.
   

**Issues not present in the dataset:**

1. **Duplicate rows** - No exact duplicates were found.  

   *Made-up example: A small 4-row dataframe with duplicates was created to demonstrate the function.


**ANALYSIS**

**Top-N Within Each Group**

In [ ]:
# Top-N within each group
# Top 3 countries for each type (Movie or TV Show) by most recent release_year
top_n = 3
top_countries = df[df['release_year'].notna()].sort_values(['type','release_year'], ascending=[True,False])
top_countries = top_countries.groupby('type').head(top_n)[['type','country','title','release_year']]
top_countries


**Bin Comparison**

In [ ]:
# Bin comparison
# Only Movies
movies_df = df[df['type'] == 'Movie'].copy()

# Create year bins
bins = [1900, 2000, 2015, 2023]
labels = ['Early', 'Middle', 'Recent']
movies_df['year_bin'] = pd.cut(movies_df['release_year'], bins=bins, labels=labels)

# Average duration per bin
avg_duration_per_bin = movies_df.groupby('year_bin', observed=False)['duration_min'].mean()
avg_duration_per_bin



**Conditional Aggregation**

In [ ]:
# Conditional aggregation
# Filter for Movies, group by rating, compute average release_year
avg_release_by_rating = df[df['type'] == 'Movie'].groupby('rating')['release_year'].mean()
avg_release_by_rating

**Percentage Distribution**

In [ ]:
# Percentage Distribution
# Percentage of total duration per type
percentage_duration = df.groupby('type')['duration_min'].sum().apply(
    lambda x: x / df['duration_min'].sum() * 100
)
percentage_duration


**Most Frequent Value by Group**

In [ ]:
# Most frequent value by group
# Group by 'type' and find the most frequent 'rating' for each
most_freq_rating = df.groupby('type')['rating'].agg(
    lambda x: x.value_counts().idxmax()  # most frequent
)
# Also get the count of that value
most_freq_count = df.groupby('type')['rating'].agg(
    lambda x: x.value_counts().max()  # count of most frequent
)

# Combine into a single dataframe
most_freq_df = pd.DataFrame({
    'Most Frequent Rating': most_freq_rating,
    'Count': most_freq_count
})

most_freq_df


**Visualization**

In [ ]:
# Histogram

plt.figure(figsize=(10,5))
sns.histplot(df['release_year'], bins=30, kde=False, color='skyblue')
plt.title('Distribution of Netflix Titles by Release Year')
plt.xlabel('Release Year')
plt.ylabel('Number of Titles')
plt.xticks(rotation=45)
plt.show()

The histogram shows that most Netflix titles were released after 2000, with a peak around 2015–2020.

**Pie Chart**

In [ ]:
plt.figure(figsize=(6,6))
df['type'].value_counts().plot.pie(autopct='%1.1f%%', colors=['lightgreen','lightcoral'])
plt.title('Percentage of Movies vs TV Shows on Netflix')
plt.ylabel('')
plt.show()


The pie chart shows that Netflix has more Movies than Tv Shows

**Line Chart**

In [ ]:
avg_duration_per_year = df[df['type'] == 'Movie'].groupby('release_year')['duration_min'].mean()

plt.figure(figsize=(10,5))
plt.plot(avg_duration_per_year.index, avg_duration_per_year.values, marker='o')
plt.title('Average Movie Duration Over the Years')
plt.xlabel('Release Year')
plt.ylabel('Average Duration (minutes)')
plt.grid(True)
plt.show()


Shows the average movie duration over the years. As we can see the average duration is decreasing.


**Box Plot**

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df[df['type'] == 'Movie'], x = 'rating', y = 'duration_min')
plt.title('Movie Duration Distribution by Rating')
plt.xlabel('Rating')
plt.ylabel('Duration (minutes)')
plt.xticks(rotation=45)
plt.show()


Shows the movie duration by their rating.

**Scatter Plot**

In [ ]:
# Scatter plot for Movies only
movies = df[df['type'] == 'Movie']

plt.figure(figsize=(8,5))
sns.scatterplot(data=movies, x='release_year', y='duration_min', hue='rating', alpha=0.7)
plt.title('Movie Duration vs Release Year by Rating')
plt.xlabel('Release Year')
plt.ylabel('Duration (minutes)')
plt.legend(title='Rating', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

Shows how movie durations vary over the years and highlights trends by rating category.

**Cleaned Dataset Upload**

In [ ]:
# adding the cleaned data
df.to_csv("diy_dataset.csv", index=False)